### 📌 Phase 5: Model Deployment with Gradio, Hugging Face Spaces & More

In this phase, we focus on deploying the fine-tuned YOLOv8 model to make it **interactive and usable** for others. The goal is to build a lightweight front-end application where users can upload X-ray images and visualize nodule detection results in real time.

The steps include:
- Creating a simple **Gradio** interface that loads the model, accepts image input, and returns the prediction with bounding boxes
- Testing the app locally to ensure it runs smoothly
- Optionally, pushing the app to **Hugging Face Spaces** to make it publicly accessible
- If needed, preparing a **FastAPI** version for more flexibility or back-end integration

This step allows us to:
- Demonstrate real-world usability of the model
- Showcase deployment skills (a key part of ML engineering)
- Provide an easy way to share results with stakeholders, clients, or recruiters

👉 **Live Demo**: [baptiste-lf-data/x-ray_Nodule_Detection](https://huggingface.co/spaces/baptiste-lf-data/x-ray_Nodule_Detection)


## ⚙️ Install Data


In [1]:
!pip install ultralytics roboflow opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127

In [2]:
from roboflow import Roboflow
from getpass import getpass

# Prompt for the key securely
api_key = getpass("Enter your Roboflow API key:")
rf = Roboflow(api_key=api_key)

project = rf.workspace("xray-chest-nodule").project("xray-chest-nodule")
dataset = project.version(7).download("yolov8")

Enter your Roboflow API key:··········
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to XRay-Chest-Nodule-7 in yolov8:: 100%|██████████| 10029/10029 [00:03<00:00, 3050.30it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
!find /content -name "data.yaml"

/content/XRay-Chest-Nodule-7/data.yaml


In [4]:
#  move 10% of train images + labels to valid
!mkdir -p /content/XRay-Chest-Nodule-7/valid/images
!mkdir -p /content/XRay-Chest-Nodule-7/valid/labels

In [5]:
import os, random, shutil

src_img = '/content/XRay-Chest-Nodule-7/train/images'
src_lbl = '/content/XRay-Chest-Nodule-7/train/labels'
dst_img = '/content/XRay-Chest-Nodule-7/valid/images'
dst_lbl = '/content/XRay-Chest-Nodule-7/valid/labels'

os.makedirs(dst_img, exist_ok=True)
os.makedirs(dst_lbl, exist_ok=True)

images = os.listdir(src_img)
random.shuffle(images)

val_split = 0.1
val_count = int(len(images) * val_split)

for img_name in images[:val_count]:
    lbl_name = img_name.replace('.jpg', '.txt').replace('.png', '.txt')
    shutil.move(os.path.join(src_img, img_name), os.path.join(dst_img, img_name))
    shutil.move(os.path.join(src_lbl, lbl_name), os.path.join(dst_lbl, lbl_name))

### Mount Drive & Prepare Save Path

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')


Mounted at /content/drive


## Gradio

### Create Gradio interface

In [6]:
%%writefile app.py

import gradio as gr
from ultralytics import YOLO
from PIL import Image, ImageDraw
import os

# Load model
model = YOLO("best.pt")
CLASS_NAMES = model.names

# Ground truth label folder
GT_LABEL_PATH = "examples/labels"

# Draw boxes on PIL image
def draw_boxes(image, boxes, labels, color, width=2):
    draw = ImageDraw.Draw(image)
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box
        draw.rectangle([x1, y1, x2, y2], outline=color, width=width)
        draw.text((x1, max(0, y1 - 10)), label, fill=color)
    return image

# Parse YOLO format label (cls cx cy w h) to box
def parse_yolo_label(txt_path, img_w, img_h):
    boxes = []
    labels = []
    if not os.path.exists(txt_path):
        return boxes, labels
    with open(txt_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, cx, cy, w, h = map(float, parts)
            x1 = int((cx - w / 2) * img_w)
            y1 = int((cy - h / 2) * img_h)
            x2 = int((cx + w / 2) * img_w)
            y2 = int((cy + h / 2) * img_h)
            boxes.append((x1, y1, x2, y2))
            labels.append(CLASS_NAMES[int(cls)])
    return boxes, labels

# Main function
def detect_nodules(input_image):
    # Convert to PIL if needed
    if not isinstance(input_image, Image.Image):
        input_image = Image.fromarray(input_image)

    # Inference
    results = model.predict(input_image, conf=0.25, verbose=False)[0]

    # Draw predictions (red)
    image_pred = input_image.copy()
    boxes_pred = []
    labels_pred = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        cls_id = int(box.cls[0])
        label = f"{CLASS_NAMES[cls_id]} {conf:.2f}"
        boxes_pred.append((x1, y1, x2, y2))
        labels_pred.append(label)
    image_pred = draw_boxes(image_pred, boxes_pred, labels_pred, color="red")

    # Try loading ground truth
    filename = getattr(input_image, "name", None)
    image_gt = input_image.copy()
    if filename:
        base = os.path.basename(filename)
        name, _ = os.path.splitext(base)
        label_file = os.path.join(GT_LABEL_PATH, name + ".txt")
        gt_boxes, gt_labels = parse_yolo_label(label_file, image_gt.width, image_gt.height)
        image_gt = draw_boxes(image_gt, gt_boxes, gt_labels, color="green")
    return image_pred, image_gt

# Generate examples dynamically
example_images = [
    ["examples/" + f]
    for f in os.listdir("examples")
    if f.endswith((".jpg", ".png", ".jpeg"))
]

# Gradio app
demo = gr.Interface(
    fn=detect_nodules,
    inputs=gr.Image(type="pil", label="Upload Chest X-ray"),
    outputs=[
        gr.Image(type="pil", label="Predicted (Red)"),
        gr.Image(type="pil", label="Ground Truth (Green)")
    ],
    title="Chest X-ray Nodule Detection with YOLOv8",
    description="Upload an X-ray image or use the examples. Red = prediction, Green = ground truth (if available).",
    examples=example_images
)

if __name__ == "__main__":
    demo.launch()


Writing app.py


### Requirements.txt

In [3]:
with open("requirements.txt", "w") as f:
    f.write("ultralytics>=8.0.0\ngradio>=4.0\nopencv-python\npillow\n")


### Folder Structure

In [4]:
# my-app/
# ├── app.py
# ├── best.pt
# ├── requirements.txt
# └── examples/
#     ├── img1003_jpg.rf.2ff865719a3c4c6397f9094ca90c9a19.jpg
#     └── labels/
#         └── img1003_jpg.rf.2ff865719a3c4c6397f9094ca90c9a19.txt

You can see my model on Hugging face here

## FastAPI

### main.py — FastAPI Inference API

In [4]:
%%writefile main.py

import gradio as gr
from ultralytics import YOLO
from PIL import Image, ImageDraw
import os

# Load YOLO model
model = YOLO("best.pt")
CLASS_NAMES = model.names

# Ground truth label folder
GT_LABEL_PATH = "examples/labels"

# Draw boxes on a PIL image
def draw_boxes(image, boxes, labels, color, width=2):
    draw = ImageDraw.Draw(image)
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box
        draw.rectangle([x1, y1, x2, y2], outline=color, width=width)
        draw.text((x1, max(0, y1 - 10)), label, fill=color)
    return image

# Convert YOLO-format label file to bounding boxes
def parse_yolo_label(txt_path, img_w, img_h):
    boxes = []
    labels = []
    if not os.path.exists(txt_path):
        return boxes, labels
    with open(txt_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, cx, cy, w, h = map(float, parts)
            x1 = int((cx - w / 2) * img_w)
            y1 = int((cy - h / 2) * img_h)
            x2 = int((cx + w / 2) * img_w)
            y2 = int((cy + h / 2) * img_h)
            boxes.append((x1, y1, x2, y2))
            labels.append(CLASS_NAMES[int(cls)])
    return boxes, labels

# Main prediction function
def detect_nodules(input_path):
    # Load image from path
    input_image = Image.open(input_path).convert("RGB")

    # Run inference
    results = model.predict(input_image, conf=0.25, verbose=False)[0]

    # Draw predicted boxes in red
    image_pred = input_image.copy()
    boxes_pred = []
    labels_pred = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        cls_id = int(box.cls[0])
        label = f"{CLASS_NAMES[cls_id]} {conf:.2f}"
        boxes_pred.append((x1, y1, x2, y2))
        labels_pred.append(label)
    image_pred = draw_boxes(image_pred, boxes_pred, labels_pred, color="red")

    # Load ground truth if available
    name = os.path.splitext(os.path.basename(input_path))[0]
    label_file = os.path.join(GT_LABEL_PATH, name + ".txt")
    gt_boxes, gt_labels = parse_yolo_label(label_file, input_image.width, input_image.height)
    image_gt = draw_boxes(input_image.copy(), gt_boxes, gt_labels, color="green")

    return image_pred, image_gt

# Example images (manually listed to ensure label match)
example_images = [
    ["examples/img1002_jpg.rf.4414eb1297a2dc6b08bbb3cd50751223.jpg"],
    ["examples/img1050_jpg.rf.4510221f7aaf9bb260413d38965945e6.jpg"],
    ["examples/img1023_jpg.rf.79c40506309b87f1086c96311e48688e.jpg"]
]

# Gradio UI
demo = gr.Interface(
    fn=detect_nodules,
    inputs=gr.Image(type="filepath", label="Upload Chest X-ray"),
    outputs=[
        gr.Image(type="pil", label="Predicted (Red Boxes)"),
        gr.Image(type="pil", label="Ground Truth (Green Boxes)")
    ],
    title="🩻 Chest X-ray Nodule Detection with YOLOv8",
    description=(
        "Upload a chest X-ray or select an example. The model detects nodules using a fine-tuned YOLOv8 model. "
        "Red = model prediction. Green = annotated ground truth (if available)."
    ),
    examples=example_images
)

if __name__ == "__main__":
    demo.launch()


### Requirements.txt

In [5]:
%%writefile requirements.txt
fastapi
uvicorn
pillow
ultralytics>=8.0.0

Overwriting requirements.txt


### Project Overview

In [4]:
# xray_api/
# ├── main.py
# ├── best.pt
# ├── requirements.txt
